In [1]:
import os
import pandas as pd
import numpy as np

from bokeh.io import output_notebook, show, push_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Select, DateRangeSlider, Span
from bokeh.layouts import layout
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
from ipywidgets import widgets
import matplotlib.pyplot as plt


os.environ['BOKEH_ALLOW_WS_ORIGIN'] = 'localhost:8888' # for the Bokeh server
output_notebook()   # enables Bokeh in Jupyter notebook

Loading BokehJS ...

# Load data

In [2]:
df_flow_rate = pd.read_feather("../notebooks/solution/temp/flow_rate_data") # Load saved data
df_flow_rate =  df_flow_rate[(df_flow_rate >= 0)] 
df_flow_rate['bf_f07_23_bahnhofstr_x'].tail()

timestamp
2021-12-31 23:47:30    14.29
2021-12-31 23:50:00    14.29
2021-12-31 23:52:30    15.04
2021-12-31 23:55:00    15.79
2021-12-31 23:57:30      NaN
Name: bf_f07_23_bahnhofstr_x, dtype: float64

# Lets filter the data just for the following specified nighttime: 02:00 - 05:00    

In [3]:
df_nighttime = df_flow_rate.between_time('02:00:00', '05:00:00') # Simple but effective filter!
df_nighttime = df_nighttime.dropna()
df_nighttime.dtypes
df_nighttime.index
df_nighttime.tail()

,bf_f07_23_bahnhofstr_x,bf_plsRKPI1102_rubpw80sbw_overflow,bf_plsRKBU1101_rub128basin_usterstr,bf_plsRKBU1102_rub128basin_overflow,bf_plsRKBA1101_rubbasin_ara_overflow,bf_plsRKBM1101_3r_rub_morg_overflow,bf_plsZUL1100_inflow_ara,bf_f08_166_luppmenweg,bf_f10_22a_bahnhofstr,bf_f02_555_mesikerstr,bf_f03_11e_russikerstr,bf_f07_23_bahnhofstr_y,bf_f12_47a_zurcherstr
timestamp,,,,,,,,,,,,,
2020-07-09 04:50:00,51.57,0.0,17.419500,0.0,0.0,0.0,48.628967,15.805,29.7790,5.596979,11.372906,51.57,32.432
2020-07-09 04:52:30,51.57,0.0,18.146000,0.0,0.0,0.0,37.576650,16.384,29.9765,5.242618,11.293873,51.57,32.432
2020-07-09 04:55:00,51.57,0.0,18.293633,0.0,0.0,0.0,24.855933,16.963,30.1740,4.888257,11.214841,51.57,32.432
2020-07-09 04:57:30,51.57,0.0,17.764600,0.0,0.0,0.0,21.992650,17.705,30.3000,5.213316,11.844987,51.57,33.361
2020-07-09 05:00:00,51.57,0.0,17.843767,0.0,0.0,0.0,25.525400,18.447,30.4260,5.538375,12.475133,51.57,34.290


In [4]:

# Create a Jupyter Dash app
app = JupyterDash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H1("Sensor Data Dashboard"),
    
    # Dropdown for selecting the sensor
    html.Label("Select sensor:"),
    dcc.Dropdown(
        id='sensor-dropdown',
        options=[{'label': sensor, 'value': sensor} for sensor in df_nighttime.columns],
        value=df_nighttime.columns[0]  # default value
    ),

    # Date range slider
    html.Label("Date Range:"),
    dcc.DatePickerRange(
        id='date-range-picker',
        start_date=df_nighttime.index.min(),
        end_date=df_nighttime.index.max(),
        display_format='MMM D, YYYY'
    ),

    # Placeholder for the histogram
    dcc.Graph(id='histogram-graph'),

    # Placeholder for the time series plot
    dcc.Graph(id='timeseries-graph'),

    # Placeholder for the derivative plot
    dcc.Graph(id='derivative-graph'),
])

# Define callback to update the histogram
@app.callback(
    Output('histogram-graph', 'figure'),
    [Input('sensor-dropdown', 'value'),
     Input('date-range-picker', 'start_date'),
     Input('date-range-picker', 'end_date')]
)
def update_histogram(selected_sensor, start_date, end_date):
    # Filter the data based on the date range and selected sensor
    mask = (df_nighttime.index >= start_date) & (df_nighttime.index <= end_date)
    filtered_data = df_nighttime.loc[mask, selected_sensor]
    
    # Create the histogram figure
    fig = go.Figure(data=[
        go.Histogram(x=filtered_data)
    ])
    fig.update_layout(title='Histogram of Mean Flow Rates', xaxis_title='mean flow rate [l/s]', yaxis_title='Frequency')
    return fig

# Define callback to update the time series plot
@app.callback(
    Output('timeseries-graph', 'figure'),
    [Input('sensor-dropdown', 'value'),
     Input('date-range-picker', 'start_date'),
     Input('date-range-picker', 'end_date')]
)
def update_timeseries(selected_sensor, start_date, end_date):
    # Filter the data based on the date range and selected sensor
    mask = (df_nighttime.index >= start_date) & (df_nighttime.index <= end_date)
    filtered_data = df_nighttime.loc[mask, selected_sensor]

    # Create the time series figure
    fig = go.Figure(data=[
        go.Scatter(x=filtered_data.index, y=filtered_data, mode='lines')
    ])
    fig.update_layout(title='Time Series of Flow Rates', xaxis_title='Time', yaxis_title='Flow Rate [l/s]')
    return fig

    return fig
# Define callback to update the derivative plot
@app.callback(
    Output('derivative-graph', 'figure'),
    [Input('sensor-dropdown', 'value'),
     Input('date-range-picker', 'start_date'),
     Input('date-range-picker', 'end_date')]
)
def update_derivative(selected_sensor, start_date, end_date):
    # Filter the data based on the date range and selected sensor
    mask = (df_nighttime.index >= start_date) & (df_nighttime.index <= end_date)
    filtered_data = df_nighttime.loc[mask, selected_sensor]
    
    # Calculate the derivative
    derivative = np.gradient(filtered_data, edge_order=2)
    
    # Create the derivative figure
    fig = go.Figure(data=[
        go.Scatter(x=filtered_data.index, y=derivative, mode='lines')
    ])
    fig.update_layout(title='Derivative of Flow Rates', xaxis_title='Time', yaxis_title='Derivative [l/s^2]')
    return fig
# Define callbacks for the time series and derivative plots similarly...

# Run the Dash app inside a Jupyter notebook
app.run_server(mode='external')


/home/pindalu/FS2024/DSIOT/smartcity-01-Patrickliuu/.venv/lib/python3.11/site-packages/dash/dash.py:550: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:8050/


My go @ Bokeh

In [6]:
from datetime import datetime
from pytz import UTC

# Create Bokeh plot for flow rates, derivatives, and histogram
p = figure(title=f"Flow Rates",
            x_axis_type="datetime",
            y_axis_label='Flow rate [l/s]', 
            height=250, #sizing_mode="stretch_width",
            tools="pan,wheel_zoom,box_zoom,reset,zoom_in,zoom_out,yzoom_in,yzoom_out",
            output_backend="webgl")


p2 = figure(title="Histogram",
            tools="",
            background_fill_color="#fafafa")
            
# === select widget to choose the evaluated sensor; needs to run in a Bokeh server
def server_doc(doc):

    def update_plot(attr, old, new):
        
        # Clear all renderers from the plots
        p.renderers.clear()
        p2.renderers.clear()


        # update title
        p.title.text = f"Flow Rates for sensor '{new}'"
        p2.title.text = f"Histogram for sensor '{new}'"

        # use ColumnDatasource
        source = ColumnDataSource(data=df_nighttime)


        # Convert milliseconds since epoch to datetime objects
        start_date = pd.to_datetime(date_range_slider.value[0], unit='ms')
        print(start_date)
        end_date = pd.to_datetime(date_range_slider.value[1], unit='ms')     
        # Absolut no idea why this slider is not working!!! nothing works at all.
        
        filtered_data = df_nighttime[(df_nighttime.index >= start_date) & (df_nighttime.index <= end_date)]
        selected_sensor_data = df_nighttime[new]
        
        # Data for HIst
        hist, edges = np.histogram(selected_sensor_data, bins=50)

        source2 = ColumnDataSource(data=dict(top=hist, left=edges[:-1], right=edges[1:]))
        # How can i update the hist data?
        
        # Add a line glyph for new sensor
        p.line(source=source, x='timestamp', y=new)
        
        # Add a hist for new sensor
        p2.quad(top='top', bottom=0, left='left', right='right', source=source2,
       fill_color="navy", line_color="white", alpha=0.5)
        # Add a Span for the mean value over the histogram

        
        #p2.add_layout(mean_line)        # Customize the figure
        p2.yaxis.axis_label = 'Frequency'
        p2.xaxis.axis_label = f'mean flow rate [l/s]'


        # Plot each night's data
        #for date, group in grouped:
            # Assuming you're only interested in a certain time window
            # e.g., between 2:00 and 4:00, adjust 'between_time' accordingly
        #    night_data = group.between_time('02:00', '04:00')
        #    p3.line(night_data.index, night_data[new], legend_label=str(date))

        # Customize the figure
        #p3.yaxis.axis_label = 'Flow rate [l/s]'
        #p3.xaxis.axis_label = 'Time'
        #p3.legend.title = 'Date'

    
        # Update the plot
        # appears not to be necessary, although it was in the original example
        #push_notebook()


    # Select widget to choose the shown sensor
    select = Select(title="Select sensor:", value='y_values', options=list(df_nighttime.columns[1:]))
    select.on_change('value', update_plot)

    # Note: There is also a date range slider
    start_date = df_nighttime.index.min()
    end_date = df_nighttime.index.max()
    date_range_slider = DateRangeSlider(start=start_date, end=end_date, value=(start_date, end_date), step=1)
    date_range_slider.on_change('value', update_plot)
    
    # Combine the plots into one layout and display
    l = layout([[select],[date_range_slider], [p], [p2]])#, [p3]]) # type: ignore
    l.sizing_mode = "stretch_width" # type: ignore
    doc.add_root(l)

    # Initial plot
    update_plot(None, None, df_nighttime.columns[1])


# Create and show the application (Bokeh server)
handler = FunctionHandler(server_doc)
app = Application(handler)
show(app, notebook_handle=True, notebook_url="http://localhost:8888")


2019-03-12 02:00:00
2019-03-12 02:00:00


In [ ]:
hist, edges = np.histogram(df_nighttime['bf_f07_23_bahnhofstr_x'].dropna(), bins=50)
df_nighttime.tail()

# I can't get any further with bokeh at the moment..

In [10]:
# Define the update_plot function to include date range filtering
def update_plot(sensor, date_range):
    plt.clf()  # Clear the current figure
    
    # Extract start_date and end_date from date_range
    start_date, end_date = date_range
    start_date = pd.to_datetime(start_date).date()
    end_date = pd.to_datetime(end_date).date()
    
    # Filter data based on the selected date range
    filtered_data = df_nighttime[sensor][(df_nighttime.index.date >= start_date) & (df_nighttime.index.date <= end_date)].dropna()

    # Plotting logic
    if not filtered_data.empty:
        mu, std = norm.fit(filtered_data)
        plt.hist(filtered_data, bins=25, alpha=0.7, density=True, edgecolor='navy')
        
        xmin, xmax = plt.xlim()
        x = np.linspace(xmin, xmax, 100)
        p = norm.pdf(x, mu, std)
        plt.plot(x, p, 'k', linewidth=2)
        
        plt.title(f'Histogram of {sensor} from {start_date} to {end_date}')
        plt.xlabel('mean flow rate [l/s]')
        plt.ylabel('Density')
        plt.show()
    else:
        print("No data available in the selected date range.")

# Create widgets
sensor_dropdown = widgets.Dropdown(
    options=df_nighttime.columns.tolist(),
    value=df_nighttime.columns[0],
    description='Sensor:',
)

date_range_slider = widgets.SelectionRangeSlider(
    options=[(date.strftime('%Y-%m-%d'), date) for date in pd.to_datetime(df_nighttime.index.date).unique()],
    index=(0, len(pd.to_datetime(df_nighttime.index.date).unique()) - 1),
    description='Date Range',
    orientation='horizontal',
    layout={'width': '500px'}
)

# Display the widgets and interactive output
widgets.interactive(update_plot, sensor=sensor_dropdown, date_range=date_range_slider)


interactive(children=(Dropdown(description='Sensor:', options=('bf_f07_23_bahnhofstr_x', 'bf_plsRKPI1102_rubpw…

<Figure size 640x480 with 0 Axes>